<a href="https://colab.research.google.com/github/agiagoulas/page-stream-segmentation/blob/master/model_training/MultiModal_Combination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive when working in Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Set working directory 

In [10]:
working_dir = "/Tobacco800/"

Imports

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!pip install fastText/.
import fasttext
from keras.callbacks import ModelCheckpoint, Callback
from keras.models import load_model, Model
from keras.optimizers import *
from keras.layers import *
from keras import regularizers
from importlib import reload
from sklearn import metrics as sklm
import numpy as np
import keras.backend as K
import tensorflow as tf
import requests

Private Imports

In [5]:
model_request = requests.get("https://raw.githubusercontent.com/agiagoulas/page-stream-segmentation/master/app/pss/model.py")
with open("model.py", "w") as f:
    f.write(model_request.text)
import model
model_img_request = requests.get("https://raw.githubusercontent.com/agiagoulas/page-stream-segmentation/master/app/pss/model_img.py")
with open("model_img.py", "w") as f:
    f.write(model_img_request.text)
import model_img

Get Fasttext word vectors

In [ ]:
if 'ft' not in locals():
    !wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
    !unzip wiki.en.zip
    ft = fasttext.load_model("wiki.en.bin")
    model.ft = ft

Set model_img properties

In [11]:
img_dim = (224,224)
model_img.img_path_template = working_dir + "Tobacco800_Small/%s.png"

Load Tobacco800 data

In [12]:
data_text_test = model.read_csv_data(working_dir + "tobacco800.test")
data_image_test = model_img.read_csv_data(working_dir + "tobacco800.test")

Load Models & Weights

In [ ]:
model_text = model.compile_model_singlepage()
model_text.load_weights(working_dir + "tobacco800_text_single-page_00.hdf5")
model_image = model_img.compile_model_singlepage(img_dim)
model_image.load_weights(working_dir + "tobacco800_image_single-page_00.hdf5")

Generate Predictions

In [ ]:
prediction_test_test = model_text.predict(model.TextFeatureGenerator(data_text_test, prevpage=False, train=False))
prediction_image_test = model_image.predict(model_img.ImageFeatureGenerator(data_image_test, img_dim, prevpage=False, train=True))
probability_text_test = np.concatenate([1 - prediction_test_test, prediction_test_test], axis = 1)
probability_image_test = np.concatenate([1 - prediction_image_test, prediction_image_test], axis = 1)

Get y_true predictions

In [20]:
_, _, _, y_true = zip(*data_text_test)
y_true = [1 if y == 'FirstPage' else 0 for y in y_true]

**Late Fusion Approach**

Get best i, j in (0,1) power normalizations for scoring

In [ ]:
max_kappa = 0
test_exponents = [x / 10 for x in range(1,11)]
for i in test_exponents:
    for j in test_exponents:
        y_predict = np.argmax(np.power(probability_text_test, i) * np.power(probability_image_test, j), axis = 1)
        acc = sklm.accuracy_score(y_true, y_predict)
        kappa = sklm.cohen_kappa_score(y_true, y_predict)
        if kappa > max_kappa:
            max_kappa = kappa
            print(str(i) + " " + str(j))
            print("Accuracy: " + str(acc))
            print("Kappa: " + str(kappa))